## Install Kaggle and download dataset

In [ ]:
# Download the GTZAN dataset
!kaggle datasets download -d andradaolteanu/gtzan-dataset-music-genre-classification

# Unzip the downloaded dataset
!unzip -q gtzan-dataset-music-genre-classification.zip

100% 1.21G/1.21G [00:59<00:00, 20.7MB/s]
100% 1.21G/1.21G [00:59<00:00, 21.9MB/s]


## Load data to pandas dataframe

In [ ]:
import pandas as pd

# Load data to dataframe
data_path = '/content/Data/features_3_sec.csv'
data = pd.read_csv(data_path)

data.head()

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,blues.00000.0.wav,66149,0.335406,0.091048,0.130405,0.003521,1773.065032,167541.630869,1972.744388,117335.771563,...,39.687145,-3.241280,36.488243,0.722209,38.099152,-5.050335,33.618073,-0.243027,43.771767,blues
1,blues.00000.1.wav,66149,0.343065,0.086147,0.112699,0.001450,1816.693777,90525.690866,2010.051501,65671.875673,...,64.748276,-6.055294,40.677654,0.159015,51.264091,-2.837699,97.030830,5.784063,59.943081,blues
2,blues.00000.2.wav,66149,0.346815,0.092243,0.132003,0.004620,1788.539719,111407.437613,2084.565132,75124.921716,...,67.336563,-1.768610,28.348579,2.378768,45.717648,-1.938424,53.050835,2.517375,33.105122,blues
3,blues.00000.3.wav,66149,0.363639,0.086856,0.132565,0.002448,1655.289045,111952.284517,1960.039988,82913.639269,...,47.739452,-3.841155,28.337118,1.218588,34.770935,-3.580352,50.836224,3.630866,32.023678,blues
4,blues.00000.4.wav,66149,0.335579,0.088129,0.143289,0.001701,1630.656199,79667.267654,1948.503884,60204.020268,...,30.336359,0.664582,45.880913,1.689446,51.363583,-3.392489,26.738789,0.536961,29.146694,blues


## Split the data

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

seed = 42

# Splitting data into features and labels
X = data.drop(['filename', 'label'], axis=1)
y = data['label']

# Encoding labels and feature normalization
encoder = LabelEncoder()
y = encoder.fit_transform(y)
X = StandardScaler().fit_transform(X)

# Split data into training and validation parts
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((7992, 58), (1998, 58), (7992,), (1998,))

## Random Forest Approach

In [ ]:
import tensorflow as tf
import numpy as np

def create_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation='softmax'),
    ])

    # Compile the model
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

models = []
for i in range(1, 11):
    print(f"\n\n### Training model {i}\n")
    model = create_model()
    model.fit(X_train, y_train, epochs=150, batch_size=32)
    model.save(f'dp_model_{i}.h5')

    models.append(model)



### Training model 1

Epoch 1/150
250/250 [==============================] - 8s 4ms/step - loss: 1.3607 - accuracy: 0.5166
Epoch 2/150
250/250 [==============================] - 1s 4ms/step - loss: 0.9196 - accuracy: 0.6873
Epoch 3/150
250/250 [==============================] - 2s 7ms/step - loss: 0.7543 - accuracy: 0.7462
Epoch 4/150
250/250 [==============================] - 2s 7ms/step - loss: 0.6296 - accuracy: 0.7888
Epoch 5/150
250/250 [==============================] - 1s 5ms/step - loss: 0.5571 - accuracy: 0.8099
Epoch 6/150
250/250 [==============================] - 2s 9ms/step - loss: 0.4787 - accuracy: 0.8426
Epoch 7/150
250/250 [==============================] - 1s 4ms/step - loss: 0.4262 - accuracy: 0.8579
Epoch 8/150
250/250 [==============================] - 1s 4ms/step - loss: 0.3748 - accuracy: 0.8775
Epoch 9/150
250/250 [==============================] - 1s 4ms/step - loss: 0.3447 - accuracy: 0.8843
Epoch 10/150
250/250 [==============================] - 1s 4ms/step

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


250/250 [==============================] - 2s 4ms/step - loss: 1.3845 - accuracy: 0.5148
Epoch 2/150
250/250 [==============================] - 1s 4ms/step - loss: 0.9411 - accuracy: 0.6799
Epoch 3/150
250/250 [==============================] - 1s 4ms/step - loss: 0.7498 - accuracy: 0.7475
Epoch 4/150
250/250 [==============================] - 1s 4ms/step - loss: 0.6463 - accuracy: 0.7857
Epoch 5/150
250/250 [==============================] - 1s 4ms/step - loss: 0.5490 - accuracy: 0.8169
Epoch 6/150
250/250 [==============================] - 1s 4ms/step - loss: 0.4819 - accuracy: 0.8406
Epoch 7/150
250/250 [==============================] - 1s 5ms/step - loss: 0.4329 - accuracy: 0.8551
Epoch 8/150
250/250 [==============================] - 2s 6ms/step - loss: 0.3799 - accuracy: 0.8710
Epoch 9/150
250/250 [==============================] - 1s 4ms/step - loss: 0.3312 - accuracy: 0.8886
Epoch 10/150
250/250 [==============================] - 1s 4ms/step - loss: 0.3042 - accuracy: 0.9012
E

In [ ]:
from sklearn.metrics import classification_report

predictions = [model.predict(X_val) for model in models]

mean_predictions = np.mean(predictions, axis=0)
ensemble_predictions = np.argmax(mean_predictions, axis=1)

report = classification_report(y_val, ensemble_predictions, target_names=encoder.classes_)
print(report)

63/63 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

       blues       0.98      0.95      0.96       208
   classical       0.94      0.98      0.96       203
     country       0.93      0.95      0.94       186
       disco       0.96      0.92      0.94       199
      hiphop       0.97      0.95      0.96       218
        jazz       0.94      0.98      0.96       192
       metal       0.98      0.99      0.98       204
         pop       0.96      0.96      0.96       180
      reggae       0.96      0.97      0.96       211
        rock       0.95      0.91      0.93       197

    accuracy                           0.96      1998
   macro avg       0.96      0.96      0.96      1998
weighted avg       0.96      0.96      0.96      1998

